### Problem 1

Consider our bacterial population growth model, where the growth rate $a(t,\omega)$ is a mean 1 Gaussian process with an exponential radial covariance function with $\sigma^2=1$ and covariance length $L=1$, with initial population $P(t=0)=1000$.

Let $a(t)$ be approximated by its 10 term K-L expansion, yielding 10 input parameters, $\xi_i$, where the $\xi_i$ are $\mathcal{N}(0,1)$.

The quantity of interest $Q(\Xi)$ is the population at time $t=1 day$.

* How sensitive is $Q(\Xi$) to the 10 K-L coefficients? (use any valid local sensitivity analysis method).
* What is the expected value of the population at $t=1$?
* Let $\xi_i=1,i=1,2,\ldots 10$.  Assuming hourly measurements of the population, construct the Fisher matrix for Q.  Are all the $X_i$ parameters identifiable?
* Compute the Sobol indices (ANOVA) for $Q(\Xi)$.
* Use a sampling based approach, combined with an sensitivity analysis results from above, to predict the probability that the population $P$ at time t=1 is greater than 4000.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as spi
from scipy import optimize

np.set_printoptions(precision = 6)

# eigenfunction and eigenvalue generators
def efunc(x, k, v_k, L=1):
    if k%2 == 0:
        return np.sin(v_k[k]*x)/np.sqrt(1-np.sin(2*v_k[k])/(2*v_k[k]))
    else:
        return np.cos(v_k[k]*x)/np.sqrt(1+np.sin(2*v_k[k])/(2*v_k[k]))

def eval(k, v_k, L=1, var_p=1):
    if k%2==0:
        return var_p*2*L/(1+L**2*v_k[k]**2)
    else:
        return var_p*2*L/(1+L**2*v_k[k]**2)

# transcendental equations
def f1(x, L=1):
    return L*x + np.tan(x)

def f2(x, L=1):
    return 1 - L*x*np.tan(x)

# vk construction
def build_vk(k):
    vk = np.zeros(k+2)
    vk[1] = optimize.root_scalar(f2, bracket = [0, np.pi/2-1E-3], method ='brentq').root
    ind = 1
    for i in range(1, k+1, 2):
        interval = [i*np.pi/2+1E-3, i*np.pi/2+np.pi-1E-3]
        ind += 1
        vk[ind] = optimize.root_scalar(f1, bracket = interval, method='brentq').root
        ind += 1
        vk[ind] = optimize.root_scalar(f2, bracket = interval, method='brentq').root
    return vk

# KL realization function
def realize(x, k, rv, vk, L, mean, var_p):
    #out = np.zeros(x.shape) + mean
    out = 0*x + mean
    
    for i in range(1, k+1):
        out += rv[i-1] * np.sqrt(eval(i, vk, L, var_p)) * efunc(x, i, vk, L)
    
    return out

# fixed realization of a given xi(omega)
def realize_a_fixed(x, k, xi, vk, L=1, mean=1, var=1):
    out = 0*x + mean   #handles non-np array input    
    
    for i in range(1, k+1):
        out += xi[i-1] * np.sqrt(eval(i, vk, L)) * efunc(x, i, vk, L)
    
    return out

# population integral
def Pop(t, xi, vk, k, P0):
    inta = spi.quad(realize_a_fixed, 0, t, args=(k, xi, vk, 1, 1, 1))[0]
    
    return P0*np.exp(inta)    

### Part 1

We make use of the analytic method for sensitivity analysis and arrive at the following gradient vector:

$$\nabla Q = P_0\text{exp}\bigg\{\bar{a}(t) + \sum\limits_{k=1}^{10}\bigg(\sqrt{\lambda_k}\xi_k(\omega)\int\limits_0^t\phi_k(t)dt\bigg)\bigg\}\bigg[\sqrt{\lambda_1}\frac{\partial}{\partial\xi_1}\big(\xi_1(\omega)\big)\int\limits_0^t\phi_1(t)dt, \dots, \sqrt{\lambda_{10}}\frac{\partial}{\partial\xi_{10}}\big(\xi_{10}(\omega)\big)\int\limits_0^t\phi_{10}(t)dt\bigg]^T$$

Substituting $t=1$, this is:

$$\nabla Q = P_0\text{exp}\bigg\{\bar{a}(1) + \sum\limits_{k=1}^{10}\bigg(\sqrt{\lambda_k}\xi_k(\omega)\int\limits_0^1\phi_k(1)dt\bigg)\bigg\}\bigg[\sqrt{\lambda_1}\int\limits_0^1\phi_1(1)dt, \dots, \sqrt{\lambda_{10}}\int\limits_0^1\phi_{10}(1)dt\bigg]^T$$

where each of the $\frac{\partial}{\partial\xi_k}\big(\xi_k(\omega)\big) = 1$. Note that we've imposed $\bar{a}(1) = 1$, the mean of the Gaussian process.

In [2]:
# generate population (this uses Prof. Carey's algorithm)
t = np.linspace(1/24, 1, 24)
popul = np.zeros(t.size)
growth = np.zeros(t.size)

k = 10 # truncating KL expansion at 10 terms
vk = build_vk(k)
xi = np.random.randn(k)
lambda_i = np.zeros(k)
int_phi_i = np.zeros(k)
grad_Q = np.zeros(k)

L = 1 # autocorrelation length
mean_gp = 1 # mean of the gaussian process
var_p = 1 # assuming a variance of 1
var = 1
P0 = 1000 # initial bacteria population

for times in range(len(t)):
    growth[times] = realize_a_fixed(t[times], k, xi, vk, L, mean_gp, var)
    popul[times] = Pop(t[times], xi, vk, k, P0)

# extract the eigenvalues and integrals of eigenfunctions to compute the gradient vector
lambda_i += [np.sqrt(eval(j, vk, L, var_p)) for j in range(1, k+1)]
# print(lambda_i)
int_phi_i += [spi.quad(efunc, 0, 1, args = (j, vk, L))[0] for j in range(1, k+1)]
# print(int_phi_i)

grad_Q += [popul[-1]*lambda_i[j-1]*int_phi_i[j-1] for j in range(1, k+1)]

# print('The growth rate at each hour is:', growth)
# print('The population at each hour is:', popul)
print('The population at t=1 is:', popul[-1])
# print('The eigenvalues are:', lambda_i)
# print('The integrated eigenfunctions are:', int_phi_i)
print('The gradient values at t=1 are', grad_Q)

The population at t=1 is: 3884.249847791632
The gradient values at t=1 are [ 2.923950e+03  1.578937e+03 -1.212478e+02  1.750666e+02  1.987458e+01
  9.553231e+01 -6.244902e+00  4.035705e+01  2.691430e+00  2.898141e+01]


### Part 2

The quantity of interest is given by:
$$Q = P(1) = P_0exp\bigg\{1 + \sum_{k=1}^{10}\sqrt{\lambda_k}\xi_k(\omega)\int_0^1\phi_k(1)dt\bigg\}$$

Let $U = 1+\sum_{k=1}^{10}\big(\sqrt{\lambda_k}\xi_k\int_0^1\phi_k(1)dt\big)$. Then, $U = \text{ln}\big(\frac{Q}{P_0}\big)$. Since the $\xi_i \sim N(0, 1)$, $U$ is a scaled sum of independent normal random variables. Thus, $Q$ is distributed log-normally with mean 1, the mean of the Gaussian process, and its variance will be the variance of the sum of 10 normally distributed random variables scaled by the eigenvalues and the integrals of the eigenfunctions:

$$\sigma_a^2 = var(U) = var\bigg(1+\sum\limits_{k=1}^{10}\bigg(\sqrt{\lambda_k}\xi_k\int_0^1\phi_k(1)dt\bigg)\bigg) \overset{ind}{=} \sum\limits_{k=1}^{10}var\bigg(\sqrt{\lambda_k}\xi_k\int_0^1\phi_k(1)dt\bigg) = \sum\limits_{k=1}^{10}\lambda_k\bigg(\int_0^1\phi_k(1)dt\bigg)^2var(\xi_k) = \sum\limits_{k=1}^{10}\lambda_k\bigg(\int_0^1\phi_k(1)dt\bigg)^2,$$ since $var(\xi_k) = 1, \forall k$.

Thus, we can write

$$\frac{Q}{P_0} \sim lognormal(1, \sigma_a^2)$$

Therefore,

$$\mathbb{E}\bigg[\frac{Q}{P_0}\bigg] = \frac{1}{P_0}\mathbb{E}[Q] = exp\bigg\{1+\frac{\sigma_a^2}{2}\bigg\} \Rightarrow \mathbb{E}[Q] = P_0exp\bigg\{1+\frac{\sigma_a^2}{2}\bigg\}$$

$$var\bigg(\frac{Q}{P_0}\bigg) = \frac{1}{P_0^2}var(Q) = (exp\{\sigma_a^2\}-1)exp\{2\mu+\sigma_a^2\} = (exp\{\sigma_a^2\}-1)exp\{2+\sigma_a^2\} \Rightarrow var(Q) = P_0^2(exp\{\sigma_a^2\}-1)exp\{2+\sigma_a^2\}$$

In [19]:
# compute the variance of a(t, omega)
var_a_i = [lambda_i[i]**2*int_phi_i[i]**2 for i in range(k)]
var_a = np.sum(var_a_i)
# print(var_a)

# compute the mean of P(1)
mean_Q = P0*np.exp(1+var_a/2)
var_Q = P0**2*(np.exp(var_a)-1)*np.exp(2+var_a)
print('The mean of the population at day 1 is', '%.6f'%(mean_Q))
print('The variance of the population at day 1 is', '%.6f'%(var_Q))
print('This implies a standard deviation of', '%.6f'%(np.sqrt(var_Q)))

The mean of the population at day 1 is 3926.913297
The variance of the population at day 1 is 16761593.206289
This implies a standard deviation of 4094.092477


### Part 3

Let $z(t) = P_0\text{exp}\bigg\{1 + \sum\limits_{k=1}^{10}\bigg(\sqrt{\lambda_k}\xi_k(\omega)\int_0^t\phi_k(t)dt\bigg)\bigg\} \overset{\xi_k = 1}{=} P_0\text{exp}\bigg\{1 + \sum\limits_{k=1}^{10}\bigg(\sqrt{\lambda_t}\int_0^t\phi_k(t)dt\bigg)\bigg\}$. Then the sensitivity matrix is:

$$\chi(q^*) = \begin{pmatrix}
z(1/24)\sqrt{\lambda_1}\int_0^{1/24}\phi_1(1/24)dt, \dots, z(1/24)\sqrt{\lambda_{10}}\int_0^{1/24}\phi_{10}(1/24)dt \\
\;\vdots \\
z(24/24)\sqrt{\lambda_1}\int_0^{24/24}\phi_1(24/24)dt, \dots, z(24/24)\sqrt{\lambda_{10}}\int_0^{24/24}\phi_{10}(24/24)dt
\end{pmatrix},$$

and the Fisher Information matrix is calculated by: $\mathcal{F} = \chi^T\chi$.

In [4]:
xi_1 = np.ones(k)
sens_mat = np.zeros(shape = (24, 10))

sens_mat += [[Pop(t[times], xi_1, vk, k, P0)*np.sqrt(eval(i, vk, L, var_p))*spi.quad(efunc, 0, t[times], args = (i, vk, L))[0] for i in range (1, k+1)] for times in range(len(t))]
        
f_mat = sens_mat.T@sens_mat

# print('The sensitivity matrix is: \n', sens_mat)
print('The Fisher Information matrix is: \n', f_mat)

The Fisher Information matrix is: 
 [[ 2.296125e+08  1.121292e+08  8.481163e+06  2.733392e+07 -4.754243e+06
   4.801721e+06  1.908981e+06  5.334558e+06 -8.856440e+05  1.844873e+06]
 [ 1.121292e+08  5.558598e+07  2.980715e+06  1.265862e+07 -2.234380e+06
   2.216663e+06  9.796881e+05  2.594050e+06 -4.654854e+05  8.426054e+05]
 [ 8.481163e+06  2.980715e+06  1.956147e+06  2.016577e+06 -3.455629e+05
   3.270441e+05  1.059398e+04  2.088087e+05  1.551340e+04  1.498500e+05]
 [ 2.733392e+07  1.265862e+07  2.016577e+06  3.930067e+06 -7.599693e+05
   5.989351e+05  2.090675e+05  6.379413e+05 -7.353990e+04  2.685187e+05]
 [-4.754243e+06 -2.234380e+06 -3.455629e+05 -7.599693e+05  2.564325e+05
  -7.981773e+03 -7.212350e+04 -1.121519e+05  1.181155e+04 -4.526358e+04]
 [ 4.801721e+06  2.216663e+06  3.270441e+05  5.989351e+05 -7.981773e+03
   2.065335e+05 -1.307738e+04  9.800377e+04 -9.854445e+03  4.734334e+04]
 [ 1.908981e+06  9.796881e+05  1.059398e+04  2.090675e+05 -7.212350e+04
  -1.307738e+04  5.917

In [6]:
eigs = np.linalg.eig(f_mat)

print('The Fisher matrix eigenvalues and corresponding eigenvectors are: \n')
for i in range(len(eigs[0])):
    print(eigs[0][i], ',', eigs[1][:, i])

The Fisher matrix eigenvalues and corresponding eigenvectors are: 

288368997.9947138 , [ 0.892209  0.436562  0.031768  0.105541 -0.018436  0.018489  0.007478
  0.020716 -0.003475  0.007108]
3056143.966510653 , [-0.134392  0.449137 -0.737113 -0.471787  0.089196 -0.062004  0.023685
 -0.007473 -0.021591 -0.037586]
293358.09487890283 , [ 0.124429 -0.169837  0.116189 -0.340783  0.6665    0.563976 -0.246753
 -0.046472  0.018366  0.019088]
67768.09626118584 , [ 0.083202 -0.150467  0.094016 -0.201846  0.225895 -0.159633  0.559719
  0.605489 -0.372665 -0.159884]
17886.969566650743 , [-0.087415  0.124546 -0.042997  0.212485 -0.133088  0.241454 -0.351783
  0.049437 -0.547087 -0.654474]
1222.1617615495709 , [ 0.148844 -0.201008  0.085834 -0.256285  0.086745 -0.320699  0.158236
 -0.40766   0.32931  -0.674054]
10.4749125065201 , [-0.19009   0.284565 -0.030631  0.342701  0.083598  0.365165  0.236371
  0.354295  0.598765 -0.29027 ]
0.022876094884936006 , [-0.195435  0.346076  0.12244   0.29906   0.37

In [7]:
epsilon = 1
chi = sens_mat

for i in range(len(eigs[0])):
    chi_sub = chi.T@chi
    eig_sub = np.linalg.eig(chi_sub)
    
    eig_vals = np.sort(eig_sub[0])
    sort_indices = [list(eig_sub[0]).index(eig_vals[i])for i in range(len(eig_vals))]
    eig_vecs = eig_sub[1][sort_indices]
    
    if eig_vals[0] < epsilon:
        max_index = np.argmax(eig_vecs[i, :])
        chi = np.delete(chi, max_index, axis = 1)
    else:
        break

print(eig_vals)

[3.474184e+01 1.821581e+03 2.315627e+04 5.761224e+04 3.018410e+06
 2.880326e+08]


At $\varepsilon = 1$, only six eigenvalues survive the algorithm. Therefore, only six parameters are identifiable. Increasing (decreasing) $\varepsilon$ will lead to fewer (more) identifiable parameters.

### Part 4

First, we compute the value of $f_0$, which should equal the mean computed analytically in part 2.

$$\begin{align}
f_0 = \mathbb{E}[Q] &= \int_{\mathbb{R}^{10}}f(\xi)\rho(\xi)d\xi \\
&= \int_{\mathbb{R}^{10}}P_0exp\bigg\{1+\sum_k\sqrt{\lambda_k}\xi_k\int_0^1\phi_kdt\bigg\}\prod_k(2\pi)^{-1/2}exp\bigg\{-\frac{1}{2}\xi_k^2\bigg\}d\xi \\
&= (2\pi)^{-10/2}P_0e\int_{\mathbb{R}^{10}}\prod_kexp\bigg\{\sqrt{\lambda_k}\xi_k\int_0^1\phi_kdt\bigg\}\prod_kexp\bigg\{-\frac{1}{2}\xi_k^2\bigg\}d\xi \\
&= (2\pi)^{-10/2}P_0e\int_{\mathbb{R}^{10}}\prod_kexp\bigg\{\sqrt{\lambda_k}\xi_k\int_0^1\phi_kdt -\frac{1}{2}\xi_k^2\bigg\}d\xi \\
&= (2\pi)^{-10/2}P_0e\prod_k\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_k}\xi_k\int_0^1\phi_kdt -\frac{1}{2}\xi_k^2\bigg\}d\xi_k\bigg]
\end{align}$$

In [8]:
# compute E(Q) = f0
def integrand(xi, lambda_i, int_phi_i):
    return np.exp(xi*lambda_i*int_phi_i - (1/2)*xi**2)

integrals = np.zeros(k)
integrals += [spi.quad(integrand, -np.inf, np.inf, args = (lambda_i[i], int_phi_i[i]))[0] for i in range(k)]

f0 = (2*np.pi)**(-10/2)*P0*np.e*np.prod(integrals)
print(f0)

3926.9132966418993


Observe that this agrees with the value calculated in part 2. Next, we compute the value of the total variance.

$$\begin{align}
D = var(Q) &= \int_{\mathbb{R}^{10}}f^2(\xi)\rho{\xi}d\xi - f_0^2 \\
&= \int_{\mathbb{R}^{10}}\bigg(P_0exp\bigg\{1+\sum_k\sqrt{\lambda_k}\xi_k\int_0^1\phi_kdt\bigg\}\bigg)^2\prod_k(2\pi)^{-1/2}exp\bigg\{-\frac{1}{2}\xi_k^2\bigg\}d\xi - f_0^2\\
&= (2\pi)^{-10/2}(P_0e)^2\int_{\mathbb{R}^{10}}\bigg(\prod_kexp\bigg\{\sqrt{\lambda_k}\xi_k\int_0^1\phi_kdt\bigg\}\bigg)^2\prod_k\bigg\{-\frac{1}{2}\xi_k^2\bigg\}d\xi - f_0^2 \\
&= (2\pi)^{-10/2}(P_0e)^2\int_{\mathbb{R}^{10}}\prod_kexp\bigg\{2\sqrt{\lambda_k}\xi_k\int_0^1\phi_kdt\bigg\}\prod_k\bigg\{-\frac{1}{2}\xi_k^2\bigg\}d\xi - f_0^2 \\
&= (2\pi)^{-10/2}(P_0e)^2\int_{\mathbb{R}^{10}}\prod_kexp\bigg\{2\sqrt{\lambda_k}\xi_k\int_0^1\phi_kdt - \frac{1}{2}\xi_k^2\bigg\}d\xi - f_0^2 \\
&= (2\pi)^{-10/2}(P_0e)^2\prod_k\bigg[\int_{\mathbb{R}}exp\bigg\{2\sqrt{\lambda_k}\xi_k\int_0^1\phi_kdt - \frac{1}{2}\xi_k^2\bigg\}d\xi_k\bigg] - f_0^2
\end{align}$$

In [9]:
# compute total variance
def int_dbl(xi, lambda_i, int_phi_i):
    return np.exp(2*xi*lambda_i*int_phi_i - (1/2)*xi**2)

integrals2 = np.zeros(k)
integrals2 += [spi.quad(int_dbl, -np.inf, np.inf, args = (lambda_i[i], int_phi_i[i]))[0] for i in range(k)]

D = (2*np.pi)**(-10/2)*(P0*np.e)**2*np.prod(integrals2) - f0**2
print(D)

16761593.206288824


Note that this agrees with the value computed in part 2.

Now, we compute each of the $f_k(q_k)$. We first derive $f_1(q_1)$, and extend this to the other nine integrals.

$$\begin{align}
\int_{\mathbb{R}^9}f(\xi)\rho(\xi_{\sim 1})d\xi_{\sim 1} &= \int_{\mathbb{R}^9}P_0exp\bigg\{1+\sum_{k=1}^{10}\sqrt{\lambda_k}\xi_k\int\limits_0^1\phi_k(1)dt\bigg\}\prod_{k \neq 1}\rho(\xi_k)d\xi_{\sim 1} \\
&= P_0e\int_{\mathbb{R}^9}exp\bigg\{\sum_{k=1}^{10}\sqrt{\lambda_k}\xi_k\int\limits_0^1\phi_k(1)dt\bigg\}\prod_{k \neq 1}\rho(\xi_k)d\xi_{\sim 1} \\
&= P_0e \int_{\mathbb{R}^9}\prod_{k=1}^{10}exp\bigg\{\sqrt{\lambda_k}\xi_k\int\limits_0^1\phi_k(1)dt\bigg\}\prod_{k \neq 1}\rho(\xi_k)d\xi_{\sim 1} \\
&= \big(P_0e\big) exp\bigg\{\sqrt{\lambda_1}\xi_1\int\limits_0^1\phi_1(1)dt\bigg\}\int_{\mathbb{R}^9}\prod_{k \neq 1}exp\bigg\{\sqrt{\lambda_k}\xi_k\int\limits_0^1\phi_k(1)dt\bigg\}\prod_{k \neq 1}\rho(\xi_k)d\xi_{\sim 1}
\end{align}$$

Introducing the probability density:
$$
\big(P_0e\big) exp\bigg\{\sqrt{\lambda_1}\xi_1\int\limits_0^1\phi_1(1)dt\bigg\}\int_{\mathbb{R}^9}\prod_{k \neq 1}exp\bigg\{\sqrt{\lambda_k}\xi_k\int\limits_0^1\phi_k(1)dt\bigg\}\prod_{k \neq 1}\rho(\xi_k)d\xi_{\sim 1} \\
\begin{align}
&= \big(P_0e\big) exp\bigg\{\sqrt{\lambda_1}\xi_1\int\limits_0^1\phi_1(1)dt\bigg\}\int_{\mathbb{R}^9}\prod_{k \neq 1}exp\bigg\{\sqrt{\lambda_k}\xi_k\int\limits_0^1\phi_k(1)dt\bigg\}\prod_{k \neq 1}(2\pi)^{-1/2}exp\bigg\{-\frac{1}{2}\xi_k^2\bigg\}d\xi_{\sim 1} \\
&= (2\pi)^{-9/2}\big(P_0e\big) exp\bigg\{\sqrt{\lambda_1}\xi_1\int\limits_0^1\phi_1(1)dt\bigg\}\int_{\mathbb{R}^9}\prod_{k \neq 1}exp\bigg\{\sqrt{\lambda_k}\xi_k\int\limits_0^1\phi_k(1)dt -\frac{1}{2}\xi_k^2\bigg\}d\xi_{\sim 1} \\
&= (2\pi)^{-9/2}\big(P_0e\big) exp\bigg\{\sqrt{\lambda_1}\xi_1\int\limits_0^1\phi_1(1)dt\bigg\}\prod_{k \neq 1}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_k}\xi_k\int\limits_0^1\phi_k(1)dt -\frac{1}{2}\xi_k^2\bigg\}d\xi_{\sim 1}\bigg],
\end{align}$$

where the last step is justified because the integrands can be written as separable products.

Since all of the $f_k(\xi_k)$ will have identical form, they can be written as:
$$f_k(\xi_k) = (2\pi)^{-9/2}\big(P_0e\big) exp\bigg\{\sqrt{\lambda_k}\xi_k\int\limits_0^1\phi_k(1)dt\bigg\}\prod_{i \neq k}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_i}\xi_i\int\limits_0^1\phi_i(1)dt -\frac{1}{2}\xi_i^2\bigg\}d\xi_{\sim k}\bigg] - f_0, \quad k = 1, \dots, 10.$$

In [10]:
p_rod = np.zeros(k)
p_rod += [xi[i]*lambda_i[i]*int_phi_i[i] for i in range(k)]
print(p_rod)
print(np.exp(p_rod))

[ 3.736830e-01 -1.483347e-01  5.420495e-02  5.912128e-02  5.701905e-03
  1.368309e-02  1.093458e-04 -4.681078e-03  2.473219e-04  3.194696e-03]
[1.453076 0.862143 1.055701 1.060904 1.005718 1.013777 1.000109 0.99533
 1.000247 1.0032  ]


In [11]:
# this computes the ten integrals f_k(xi_k)
fk_xi = np.zeros(k)
fk_xi += [(2*np.pi)**(-9/2)*P0*np.e*np.exp(p_rod[i])*np.prod(np.delete(integrals, i)) - f0 for i in range(k)]
    
print(fk_xi)

[ 3.713232e+02 -8.098260e+02  2.167135e+02  2.349350e+02  2.240314e+01
  5.289772e+01  4.243392e-01 -1.855019e+01  9.703890e-01  1.245570e+01]


Next, we compute the partial variances for the individual $KL$ terms and calculate their Sobol indices.

$$\begin{align}
D_k &= \int_{\mathbb{R}}f_k^2(\xi_k)\rho(\xi_k)d\xi_k \\
&= \int_{\mathbb{R}}\bigg[\int_{\mathbb{R}^9}f(\xi)\rho(\xi_{\sim k})d\xi_{\sim k}\bigg]^2\rho(\xi_k)d\xi_k - f_0^2,
\end{align}$$

which is taken from page 28 in the lecture notes. To simplify, again consider the first integral:

$$\begin{align}
D_1 &= \int_{\mathbb{R}}\Bigg((2\pi)^{-9/2}\big(P_0e\big)exp\bigg\{\sqrt{\lambda_1}\xi_1\int_0^1\phi_1(1)dt\bigg\}\prod_{k \neq 1}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_k}\xi_k\int_0^1\phi_k(1)dt - \frac{1}{2}\xi_k^2\bigg\}d\xi_{\sim k}\bigg]\Bigg)^2 \rho(\xi_1)d\xi_1 - f_0^2\\
&= (2\pi)^{-19/2}\big(P_0e\big)^2\int_{\mathbb{R}}exp\bigg\{2\sqrt{\lambda_1}\xi_1\int_0^1\phi_1(1)dt - \frac{1}{2}\xi_1^2\bigg\}d\xi_1\Bigg(\prod_{k \neq 1}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_k}\xi_k\int_0^1\phi_k(1)dt - \frac{1}{2}\xi_k^2\bigg\}d\xi_{\sim k}\bigg]\Bigg)^2 - f_0^2,
\end{align}$$

and the remaining nine integrals will have the same form.

$$D_k = (2\pi)^{-19/2}\big(P_0e\big)^2\int_{\mathbb{R}}exp\bigg\{2\sqrt{\lambda_k}\xi_k\int_0^1\phi_k(1)dt - \frac{1}{2}\xi_k^2\bigg\}d\xi_k\Bigg(\prod_{i \neq k}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_i}\xi_i\int_0^1\phi_i(1)dt - \frac{1}{2}\xi_i^2\bigg\}d\xi_{\sim i}\bigg]\Bigg)^2 - f_0^2$$

In [12]:
# partial variances
D_k = np.zeros(k)
D_k += [(2*np.pi)**(-19/2)*(P0*np.e)**2*integrals2[i]*(np.prod(np.delete(integrals, i)))**2 - f0**2 for i in range(k)]
print("The partial variances are", D_k)

# compare sum of partial variances with total variance
print("The sum of the partial variances is", np.sum(D_k))
print("The total variance is", D)
print("The share of the total variance made up by the partial variances is", '%.4f'%(np.sum(D_k)/D))

# compute Sobol indices
sobol_i = np.zeros(k)
sobol_i += [D_k[i]/D for i in range(k)]
print("The Sobol indices are \n", sobol_i)

The partial variances are [1.175636e+07 2.770724e+06 1.503307e+04 3.135712e+04 4.037290e+02
 9.330829e+03 3.986026e+01 1.664756e+03 7.403801e+00 8.584983e+02]
The sum of the partial variances is 14585776.10086842
The total variance is 16761593.206288824
The share of the total variance made up by the partial variances is 0.8702
The Sobol indices are 
 [7.013866e-01 1.653020e-01 8.968760e-04 1.870772e-03 2.408655e-05
 5.566791e-04 2.378071e-06 9.931966e-05 4.417122e-07 5.121818e-05]


Next, we compute the partial variances for the $KL$ cross-terms and calculate their Sobol indices. We begin with $f_{kl}(\xi_k,\xi_l)$ and then calculate $D_{kl}$.

$$\begin{align}
f_{kl}(\xi_k,\xi_l) &= \int_{\mathbb{R}^8}f(\xi)\rho(\xi_{\sim kl})d\xi_{\sim kl} - f_k(\xi_k) - f_l(\xi_l) - f_0 \\
&= \int_{\mathbb{R}^8}P_0exp\bigg\{1+\sum_k\sqrt{\lambda_k}\xi_k\int_0^1\phi_k(1)dt\bigg\}\prod_{i \neq k,l}(2\pi)^{-1/2}exp\bigg\{-\frac{1}{2}\xi_i^2\bigg\}d\xi_{\sim k,l} - f_k(\xi_k) - f_l(\xi_l) - f_0 \\
&= (2\pi)^{-8/2}P_0e\int_{\mathbb{R}^8}\prod_kexp\bigg\{\sqrt{\lambda_k}\xi_k\int_0^1\phi_k(1)dt\bigg\}\prod_{i \neq k,l}exp\bigg\{-\frac{1}{2}\xi_k^2\bigg\}d\xi_{\sim k,l} - f_k(\xi_k) - f_l(\xi_l) - f_0 \\
&= (2\pi)^{-8/2}P_0e\prod_{1,2}exp\bigg\{\sqrt{\lambda_{1,2}}\xi_{1,2}\int_0^1\phi_{1,2}(1)dt\bigg\}\int_{\mathbb{R}^8}\prod_{k \neq 1,2}exp\bigg\{\sqrt{\lambda_k}\xi_k\int_0^1\phi_k(1)dt-\frac{1}{2}\xi_k^2\bigg\}d\xi_{\sim 1,2} - f_k(\xi_k) - f_l(\xi_l) - f_0 \\
&= (2\pi)^{-8/2}P_0exp\bigg\{1+\sum_{i=1,2}\sqrt{\lambda_i}\xi_i\int_0^1\phi_i(1)dt\bigg\}\prod_{k \neq 1,2}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_k}\xi_k\int_0^1\phi_k(1)dt-\frac{1}{2}\xi_k^2\bigg\}d\xi_{\sim 1,2}\bigg] - f_k(\xi_k) - f_l(\xi_l) - f_0 \\
\Rightarrow f_{kl}(\xi_k,\xi_l) &= (2\pi)^{-8/2}P_0exp\bigg\{1+\sum_{i=k,l}\sqrt{\lambda_i}\xi_i\int_0^1\phi_i(1)dt\bigg\}\prod_{i \neq k,l}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_i}\xi_i\int_0^1\phi_i(1)dt-\frac{1}{2}\xi_i^2\bigg\}d\xi_{\sim k,l}\bigg] - f_k(\xi_k) - f_l(\xi_l) - f_0
\end{align}$$

In [13]:
# this computes the f_kl(xi_kl)
fkl_xi = np.zeros(shape=(k, k))
fkl_xi += [[(2*np.pi)**(-8/2)*P0*np.e*np.exp(np.sum([p_rod[i],p_rod[j]]))*np.prod(np.delete(integrals,[i,j]))-fk_xi[i]-fk_xi[j]-f0 for j in range(k)] for i in range(k)]

# print(fkl_xi)

$$D_{kl} = \int\int\limits_{\mathbb{R}^2}f_{kl}^2(\xi_k,\xi_l)\rho(\xi_k)\rho(\xi_l)d\xi_kd\xi_l$$

To simplify this, let $z_{kl} = (2\pi)^{-8/2}P_0exp\big\{1+\sum_{i=k,l}\sqrt{\lambda_i}\xi_i\int_0^1\phi_i(1)dt\big\}\prod_{i \neq k,l}\big[\int_{\mathbb{R}}exp\big\{\sqrt{\lambda_i}\xi_i\int_0^1\phi_i(1)dt - \frac{1}{2}\xi_i^2\big\}d\xi_{\sim k,l}\big]$, then

$$\begin{align}
f_{kl}^2(\xi_k,\xi_l) &= (z_{kl} - f_k - f_l - f_0)^2 \\
&= z_{kl}^2 + f_k^2 + f_l^2 + f_0^2 - 2f_kz_{kl} - 2f_lz_{kl} - 2f_0z_{kl} + 2f_kf_l + 2f_0f_k + 2f_0f_l
\end{align}$$

Then $D_{kl}$ can be written as

$$\int\int\limits_{\mathbb{R}^2}z_{kl}^2\rho_k\rho_ld\xi_kd\xi_l + \int\int\limits_{\mathbb{R}^2}f_k^2\rho_k\rho_ld\xi_kd\xi_l + \int\int\limits_{\mathbb{R}^2}f_l^2\rho_k\rho_ld\xi_kd\xi_l + \int\int\limits_{\mathbb{R}^2}f_0^2\rho_k\rho_ld\xi_kd\xi_l - 2\int\int\limits_{\mathbb{R}^2}f_kz_{kl}\rho_k\rho_ld\xi_kd\xi_l - 2\int\int\limits_{\mathbb{R}^2}f_lz_{kl}\rho_k\rho_ld\xi_kd\xi_l - 2\int\int\limits_{\mathbb{R}^2}f_0z_{kl}\rho_k\rho_ld\xi_kd\xi_l + 2\int\int\limits_{\mathbb{R}^2}f_kf_l\rho_k\rho_ld\xi_kd\xi_l + 2\int\int\limits_{\mathbb{R}^2}f_0f_k\rho_k\rho_ld\xi_kd\xi_l + 2\int\int\limits_{\mathbb{R}^2}f_0f_l\rho_k\rho_ld\xi_kd\xi_l,$$

each of which can be simplified.

Terms 1-4:

$$\int\int\limits_{\mathbb{R}^2}z_{kl}^2\rho_k\rho_ld\xi_kd\xi_l = (2\pi)^{-9}(P_0e)^2\bigg(\prod_{i \neq k,l}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_i}\xi_i\int_0^1\phi_idt - \frac{1}{2}\xi_i^2\bigg\}d\xi_{\sim k,l}\bigg]\bigg)^2\int_{\mathbb{R}}exp\bigg\{2\sqrt{\lambda_k}\xi_k\int_0^1\phi_kdt - \frac{1}{2}\xi_k^2\bigg\}d\xi_k\int_{\mathbb{R}}exp\bigg\{2\sqrt{\lambda_l}\xi_l\int_0^1\phi_ldt - \frac{1}{2}\xi_l^2\bigg\}d\xi_l$$

-------------------------------------------------------------------------------------------------------------------------------

$$\int\int\limits_{\mathbb{R}^2}f_k^2\rho_k\rho_ld\xi_kd\xi_l = \int_{\mathbb{R}}\rho_ld\xi_l\int_{\mathbb{R}}f_k^2\rho_kd\xi_k = D_k$$

-------------------------------------------------------------------------------------------------------------------------------

$$\int\int\limits_{\mathbb{R}^2}f_l^2\rho_k\rho_ld\xi_kd\xi_l = \int_{\mathbb{R}}\rho_kd\xi_k\int_{\mathbb{R}}f_l^2\rho_ld\xi_l = D_l$$

-------------------------------------------------------------------------------------------------------------------------------

$$\int\int\limits_{\mathbb{R}^2}f_0^2\rho_k\rho_ld\xi_kd\xi_l = f_0^2\int_{\mathbb{R}}\rho_kd\xi_k\int_{\mathbb{R}}\rho_ld\xi_l = f_0^2$$ because $$\int_{\mathbb{R}}\rho_kd\xi_k = \int_{\mathbb{R}}\rho_ld\xi_l = 1$$

Observe that 
$$\begin{align}
\int_{\mathbb{R}}z_{kl}\rho_ld\xi_l &= (2\pi)^{-9/2}(P_0e)\int_{\mathbb{R}}exp\bigg\{\sum_{i=k,l}\sqrt{\lambda_i}\xi_i\int_0^1\phi_i(1)dt\bigg\}\prod_{i \neq k,l}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_i}\xi_i\phi_i(1)dt - \frac{1}{2}\xi_i^2\bigg\}d\xi_{\sim k,l}\bigg]exp\bigg\{-\frac{1}{2}\xi_l^2\bigg\}d\xi_l \\
&= (2\pi)^{-9/2}(P_0e)\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_k}\xi_k\int_0^1\phi_k(1)dt\bigg\}exp\bigg\{\sqrt{\lambda_l}\xi_l\int_0^1\phi_l(1)dt-\frac{1}{2}\xi_l^2\bigg\}\prod_{i \neq k,l}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_i}\xi_i\phi_i(1)dt - \frac{1}{2}\xi_i^2\bigg\}d\xi_{\sim k,l}\bigg]d\xi_l \\
&= (2\pi)^{-9/2}(P_0e)exp\bigg\{\sqrt{\lambda_k}\xi_k\int_0^1\phi_k(1)dt\bigg\}\prod_{i \neq k}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_i}\xi_i\phi_i(1)dt - \frac{1}{2}\xi_i^2\bigg\}d\xi_{\sim k}\bigg] \\
&= z_k = f_k + f_0 \\
\Rightarrow 2\int\int_{\mathbb{R}^2}f_kz_{kl}\rho_k\rho_ld\xi_kd\xi_l &= 2\int_{\mathbb{R}}f_k(f_k+f_0)\rho_kd_k \\
&= 2\bigg[\int_{\mathbb{R}}f_k^2\rho_kd\xi_k + f_0\int_{\mathbb{R}}f_k\rho_kd_k\bigg] \\
&= 2D_k,
\end{align}$$

because $$\int_{\mathbb{R}}f_k\rho_kd_k = 0$$ by constraint. Similarly,

$$2\int\int_{\mathbb{R}^2}f_lz_{kl}\rho_k\rho_ld\xi_kd\xi_l = 2D_l$$

-------------------------------------------------------------------------------------------------------------------------------

Next,

$$\begin{align}
2\int\int_{\mathbb{R}^2}f_0z_{kl}\rho_k\rho_ld\xi_kd\xi_l &= 2(2\pi)^{-10/2}(P_0e)f_0\prod_{i \neq k,l}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_i}\xi_i\int_0^1\phi_idt - \frac{1}{2}\xi_i^2\bigg\}d\xi_{\sim k,l}\bigg]\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_k}\xi_k\phi_k(1)dt-\frac{1}{2}\xi_k^2\bigg\}d\xi_k\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_l}\xi_l\phi_l(1)dt-\frac{1}{2}\xi_l^2\bigg\}d\xi_l \\
&= 2(2\pi)^{-10/2}(P_0e)f_0\prod_{i}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_i}\xi_i\int_0^1\phi_idt - \frac{1}{2}\xi_i^2\bigg\}d\xi_i\bigg] \\
&= 2f_0^2
\end{align}$$

Terms 8-10:

$$2\int\int\limits_{\mathbb{R}^2}f_kf_lz_{kl}\rho_k\rho_ld\xi_kd\xi_l = 2\int_{\mathbb{R}}f_k\rho_kd\xi_k\int_{\mathbb{R}}f_l\rho_ld\xi_l = 0$$ because $$\int_{\mathbb{R}}f_k\rho_kd\xi_k = \int_{\mathbb{R}}f_l\rho_ld\xi_l = 0$$ by constraint.

-------------------------------------------------------------------------------------------------------------------------------
Similarly,

$$2\int\int\limits_{\mathbb{R}^2}f_0f_k\rho_k\rho_ld\xi_kd\xi_l = 2f_0\int_{\mathbb{R}}\rho_ld\xi_k\int_{\mathbb{R}}f_k\rho_kd\xi_k = 0,$$

and

$$2\int\int\limits_{\mathbb{R}^2}f_0f_l\rho_k\rho_ld\xi_kd\xi_l = 2f_0\int_{\mathbb{R}}\rho_kd\xi_k\int_{\mathbb{R}}f_l\rho_ld\xi_l = 0.$$

Therefore, $$D_{kl} = (2\pi)^{-9}(P_0e)^2\bigg(\prod_{i \neq k,l}\bigg[\int_{\mathbb{R}}exp\bigg\{\sqrt{\lambda_i}\xi_i\int_0^1\phi_idt - \frac{1}{2}\xi_i^2\bigg\}d\xi_{\sim k,l}\bigg]\bigg)^2\int_{\mathbb{R}}exp\bigg\{2\sqrt{\lambda_k}\xi_k\int_0^1\phi_kdt - \frac{1}{2}\xi_k^2\bigg\}d\xi_k\int_{\mathbb{R}}exp\bigg\{2\sqrt{\lambda_l}\xi_l\int_0^1\phi_ldt - \frac{1}{2}\xi_l^2\bigg\}d\xi_l - D_k - D_l - f_0^2$$

In [14]:
# cross term partial variances
D_kl = np.zeros(shape=(k, k))
D_kl += [[(2*np.pi)**(-9)*(P0*np.e)**2*np.prod(np.delete(integrals, [i,j]))**2*integrals2[i]*integrals2[j]-D_k[i]-D_k[j]-f0**2 for j in range(k)] for i in range(k)]

# 
print('The cross-term partial variances are \n', D_kl)

# compare sum of partial variances with total variance
partial_sum = np.sum(np.triu(D_kl)) - np.trace(D_kl)
print('The sum of the cross-term partial variances is', partial_sum)
print('The share of the total variance made up by the partial variances is', '%.4f'%(partial_sum/D))
print('The sum of the computed partial variances relative to the total variance is', '%.4f'%((np.sum(D_k)+partial_sum)/D))

# compute Sobol indices
sobol_ij = np.zeros(shape=(k, k))
sobol_ij += [[D_kl[i,j]/D for j in range(k)] for i in range(k)]
print("The Sobol indices are \n", sobol_ij)

The cross-term partial variances are 
 [[4.914372e+08 2.112338e+06 1.146088e+04 2.390596e+04 3.077940e+02
  7.113615e+03 3.038857e+01 1.269173e+03 5.644492e+00 6.544999e+02]
 [2.112338e+06 1.381016e+08 2.701086e+03 5.634130e+03 7.254052e+01
  1.676528e+03 7.161942e+00 2.991171e+02 1.330287e+00 1.542518e+02]
 [1.146088e+04 2.701086e+03 8.172372e+07 3.056900e+01 3.935818e-01
  9.096311e+00 3.885842e-02 1.622914e+00 7.217709e-03 8.369211e-01]
 [2.390596e+04 5.634130e+03 3.056900e+01 8.199970e+07 8.209628e-01
  1.897377e+01 8.105385e-02 3.385198e+00 1.505526e-02 1.745713e+00]
 [3.077940e+02 7.254052e+01 3.935818e-01 8.209628e-01 8.147694e+07
  2.442911e-01 1.043584e-03 4.358508e-02 1.938380e-04 2.247640e-02]
 [7.113615e+03 1.676528e+03 9.096311e+00 1.897377e+01 2.442911e-01
  8.162747e+07 2.411891e-02 1.007322e+00 4.479945e-03 5.194659e-01]
 [3.038857e+01 7.161942e+00 3.885842e-02 8.105385e-02 1.043586e-03
  2.411891e-02 8.147081e+07 4.303167e-03 1.913868e-05 2.219100e-03]
 [1.269173e+03 2

### Part 5

In [15]:
# generate population samples
sample = 1000
t0 = np.linspace(1/24, 1, 24)
popul0 = np.zeros(shape = (sample,t.size))
growth0 = np.zeros(shape = (sample,t.size))

k0 = 6 # truncating KL expansion at 6 terms based on results of sensitivity analysis
vk0 = build_vk(k0)

In [16]:
for j in range(popul0.shape[0]):
    xi0 = np.random.randn(k0)
    lambda_i0 = np.zeros(k0)
    int_phi_i0 = np.zeros(k0)
    for times in range(len(t0)):
        popul0[j][times] = Pop(t0[times], xi0, vk0, k0, P0)

In [ ]:
# print(popul0)

In [17]:
taco = 0

for i in range(popul0.shape[0]):
    if popul0[i, -1] > 4000:
        taco += 1

print(taco)
print('The probability of the population after one day being greater than 4000 is', '%.4f'%(taco/sample))

324
The probability of the population after one day being greater than 4000 is 0.3240
